See [circular_exp_fitting](circular_exp_fitting.ipynb)  and  [sersic_integral_experiments](sersic_integral_experiments.ipynb) for prior art

In [81]:
import numpy as np

from scipy import special

import sympy

import regions

In [82]:
from matplotlib import pyplot as plt
%matplotlib inline

In [83]:
TWOPI = 2*np.pi
ICOEFFS = [2, -1/3, 4/405, 46/25515, 131/1148175, -2194697/30690717750]

def f_bn(n):
    """
    Ciotti and Bertin 99, valid for n >~ 0.36
    """
    return np.sum([C*n**(1-i) for i, C in enumerate(ICOEFFS)], axis=0)

def sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return Ie * np.exp(-bn*((R/Re)**(1/n)-1))

def log_sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return np.log(Ie) - bn*((R/Re)**(1/n)-1)

Now implement technique 5 for the sersic integral from sersic_integral_experiments

In [84]:
def make_grid_Iweighted(nr, nphi, outerr, Re, n):
    bn_val = f_bn(n)

    gamma2 = special.gammainc(2*n, outerr**(1/n) * Re**(-1/n) * bn_val)

    u = np.linspace(0, 1, nr)
    r = Re * (special.gammaincinv(2*n, u * gamma2) / bn_val)**n
    phi = np.linspace(0, 2*np.pi, nphi+1)[:-1]

    return np.meshgrid(r, phi)

def sersic_integral(Re, n, region, x0=0, y0=0, nr=100, nphi=100):
    """
    Note the the region, Re, x0, and y0 should all be in pixels
    """

    outerr = np.max(np.abs(region.bounding_box.extent))
    outer_integ = sersic_integral(outerr, Re, n)

    r, phi = make_grid_Iweighted(nr, nphi, outerr, Re, n)
    xg = r*np.cos(phi) + x0
    yg = r*np.sin(phi) + y0
    msk = region.contains(regions.PixCoord(x=xg, y=yg))

    frac_in_grid = np.sum(msk)/msk.size

    return outer_integ * frac_in_grid

Not clear if we should be sampling from the areal sersic function or the sersic function itself.  Fortunately they are very similar:

In [ ]:
R, Ie, Re, n, bn, R1 = sympy.symbols('R, I_e, R_e, n, b_n, R_1', real=True, positive=True)
sersic = Ie * sympy.exp(-bn*((R/Re)**(1/n)-1))

R1, R2 = sympy.symbols('R1, R2', real=True, positive=True)
normalized_integral = sympy.simplify(sympy.integrate(sersic, (R,0, R1)) / sympy.integrate(sersic, (R,0, R2)))
normalized_integral

Which is the same as areal except it's $n$ instead of $2n$

In [86]:
def get_sersic_r(npoints, outerr, Re, n, rstate=np.random.default_rng(), areal=False):
    bn_val = f_bn(n)

    gammaparam = 2*n if areal else n

    gamma2 = special.gammainc(gammaparam, outerr**(1/n) * Re**(-1/n) * bn_val)

    u = rstate.uniform(size=npoints)
    r = Re * (special.gammaincinv(gammaparam, u * gamma2) / bn_val)**n

    return r

In [ ]:

plt.hist(get_sersic_r(100000, 3, 1, 1.5, areal=True), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, np.inf, 1, 1.5, areal=True), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, 3, .5, 1.5, areal=True), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, 3, 1, 4., areal=True), bins='auto', histtype='step', density=True, log=True)
plt.xlim(0, 5);
plt.ylim(10**-2.5, 2);

In [ ]:
plt.hist(get_sersic_r(100000, 3, 1, 1.5, areal=False), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, np.inf, 1, 1.5, areal=False), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, 3, .5, 1.5, areal=False), bins='auto', histtype='step', density=True, log=True)
plt.hist(get_sersic_r(100000, 3, 1, 4., areal=False), bins='auto', histtype='step', density=True, log=True)
plt.xlim(0, 5);

In [ ]:
def generate_unif_box(n=None, halfsize=1, density=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        A = (halfsize*2)**2
        n = density * A
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    x = rstate.uniform(-halfsize, halfsize, n)
    y = rstate.uniform(-halfsize, halfsize, n)

    return x, y

def uniform_in_region(density, region, rstate=np.random.RandomState()):
    maxpx = np.max(region.bounding_box.extent)
    xs, ys = generate_unif_box(halfsize=maxpx, density=density, rstate=rstate)
    msk = region.contains(PixCoord(xs, ys))

    return xs[msk], ys[msk]

def expinteg(F, Re, b=BVAL):
    sc = np.isscalar(F)
    res = np.atleast_1d(-Re*(lambertw((F-1)/np.exp(1), -1).real + 1)/b)
    res.ravel()[np.isnan(res.ravel())] = 0
    return res[0] if sc else res
    
def generate_exp_dglx(n=None, Re=1, density=None, region=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        # density is the mean density within the HLR
        hA = np.pi*Re**2
        n = 2* density * hA
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    U = rstate.uniform(0, 1, n)
    r = expinteg(U, Re)
    phi = rstate.uniform(0, 2*np.pi, n)

    x = np.cos(phi)*r
    y = np.sin(phi)*r

    if region is not None:
        msk = region.contains(PixCoord(x, y))
        x = x[msk]
        y = y[msk]

    return x, y

In [5]:
def glx_and_uniform(glx_hdensity, bkg_density, region, Re=1, rstate=np.random.RandomState()):
    g = generate_exp_dglx(density=glx_hdensity, Re=Re, region=region, rstate=rstate)
    b = uniform_in_region(bkg_density, region=region, rstate=rstate)

    return np.concatenate((g, b), axis=1), g, b

In [ ]:
offset_region1 = RectanglePixelRegion(PixCoord(2, 3), 2, 3)
offset_region2 = RectanglePixelRegion(PixCoord(0, -1), 6, 6)
poly_region = PolygonPixelRegion(PixCoord([-3, -3, 3, 3, 1, 1, -3], [2, -4, -4, 5, 5, 3, 2]))

offset_region1.plot()
offset_region2.plot()
poly_region.plot(color='red')
plt.xlim(-10, 10)
plt.ylim(-10, 10)

In [7]:
circular_large_reg = CirclePixelRegion(PixCoord(0, 0), 10)
circular_small_reg = CirclePixelRegion(PixCoord(0, 0), 3)
rectangular_large_reg = RectanglePixelRegion(PixCoord(0, 0), 20, 20)
rectangular_small_reg = RectanglePixelRegion(PixCoord(0, 0), 6, 6)

offset_region1 = RectanglePixelRegion(PixCoord(2, 3), 2, 3)
offset_region2 = RectanglePixelRegion(PixCoord(0, -1), 6, 6)
offset_regions = offset_region1 | offset_region2

offset_poly_region = PolygonPixelRegion(PixCoord([-3, -3, 3, 3, 1, 1, -3], [2, -4, -4, 5, 5, 3, 2]))

all_regions = {nm: globals()[nm] for nm in 'circular_large_reg,circular_small_reg,rectangular_large_reg,rectangular_small_reg,offset_poly_region'.split(',')}

# Set up fitting code

`region` is a fixed region - if None the integrals are implied to infity.

Parameters:
* x0, y0 -> center
* Re -> half-light radius
* fbkg -> background fraction of all the stars

In [8]:
import dynesty
from dynesty import plotting as dyplot

param_names = 'x0, y0, Re, fbkg'.split(', ')

Derive the bounded scale-free prior generator

In [ ]:
σ, x, a, b, A, F = sympy.symbols('σ,x,a,b,A,F', real=True, positive=True)
Asoln = sympy.solve(sympy.integrate(A/σ,(σ,a,b)) - 1, A)[0]
sympy.solve(sympy.integrate(A/σ,(σ,a,x)).subs(A, Asoln) - F, x)[0]

Prior below is that x0, y0 are on U[-1, 1], Re is scale-free on [.01, 100], fbkg is U[.1, .9]

In [10]:
uppri = 100
lpri = .01

def prior_transform(u):
    x = np.empty_like(u)
    x[:2] = u[:2]*2 -1
    x[2] = lpri**(1-u[2]) * uppri**u[2]
    x[3] = u[3]*.8 + .1

    return x

In [ ]:
test_region = circular_large_reg
test_data, tg, tb = glx_and_uniform(25, 2, test_region)
truths = [0, 0, 1, -1]

truths[-1] = len(tb[0])/(len(tb[0]) + len(tg[0]))

len(tg[0]), len(tb[0]), truths[-1]

In [12]:
LTWOPI = np.log(TWOPI)

def loglike(p, data, area):
    x0, y0, Re, fbkg = p
    x, y = data
    α = Re/BVAL

    r = np.hypot(x-x0, y-y0)
    lgal = np.log(1-fbkg) - 2*np.log(α) - r/α - LTWOPI
    lbkg = np.log(fbkg/area)
    ll = np.logaddexp(lgal, lbkg)

    return np.sum(ll)
    

In [ ]:
with dynesty.pool.Pool(16, loglike, prior_transform,
                                logl_kwargs={'data': test_data, 'area':test_region.area},
                      ) as pool:
    dsampler = dynesty.DynamicNestedSampler(pool.loglike, pool.prior_transform, pool=pool,
                                            ndim=len(param_names),sample='rslice')
    dsampler.run_nested(maxiter=100000,  wt_kwargs={'pfrac': 1.0})

In [ ]:
fig, axes = dyplot.runplot(dsampler.results)

span = [1,1, .95, .95]

fig, axes = dyplot.traceplot(dsampler.results, labels=param_names, truths=truths,span=span)
fig, axes = dyplot.cornerplot(dsampler.results, labels=param_names, truths=truths,span=span)
fig, axes = dyplot.cornerpoints(dsampler.results, labels=param_names, truths=truths)

In [ ]:
results = {}
truths = {}
for regnm, test_region in all_regions.items():
    print(regnm)
    
    test_data, tg, tb = glx_and_uniform(25, 2, test_region)
    truths[regnm] = [0, 0, 1, len(tb[0])/(len(tb[0]) + len(tg[0]))]

    with dynesty.pool.Pool(16, loglike, prior_transform,
                                logl_kwargs={'data': test_data, 'area':test_region.area},
                      ) as pool:
        dsampler = dynesty.DynamicNestedSampler(pool.loglike, pool.prior_transform, pool=pool,
                                                ndim=len(param_names),sample='rslice')
        dsampler.run_nested(maxiter=100000,  wt_kwargs={'pfrac': 1.0})
    results[regnm] = dsampler.results

In [ ]:
for nm, res in results.items():
    fig, axes = dyplot.cornerplot(res, labels=param_names, truths=truths[nm])
    axes[0][0].text(.7,.8, nm, transform=fig.transFigure)